# Mixed - Scenario 1~4

In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

from statsmodels.stats.anova import anova_lm 
import statsmodels.formula.api as smf

required_packages = ['base', 'forecast', 'CondIndTests','RCIT'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

In [2]:
r = robjects.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')
RCIT = importr('RCIT')
pcalg = importr('pcalg')

In [3]:
def IgnorTest(cause=None, effect=None, explorer=None, covariate=None, level=0.05, dtype="continuous"): #inputs are arrays, dtypes are continuous/binary/mixed
    tmp_cause=pd.DataFrame({'A': cause})
    tmp_effect=pd.DataFrame({'Y': effect})
    tmp_explorer=pd.DataFrame({'V': explorer})
    tmp_covariate = pd.DataFrame(covariate)
    dim_covariate = tmp_covariate.shape[1]
    
    if dtype=="continuous":
        if dim_covariate==0:
            tmp_conditioning=tmp_cause
        else:
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_conditioning = ro.conversion.py2rpy(tmp_conditioning)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_effect = ro.conversion.py2rpy(tmp_effect)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_explorer = ro.conversion.py2rpy(tmp_explorer)
            
        sample_size = r.unlist(r.dim(r_from_pd_conditioning))[0]
        dim = r.unlist(r.dim(r_from_pd_conditioning))[1]
            
        r_from_pd_explorer=r.unlist(r_from_pd_explorer)    
        r_from_pd_effect=r.unlist(r_from_pd_effect)   
        r_from_pd_conditioning=r.unlist(r_from_pd_conditioning)   
        r_from_pd_conditioning=r.matrix(r_from_pd_conditioning,sample_size,dim)
        
        #test=r.CondIndTest(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning, method = "KCI")
        #pvalue=r.unlist(test)[3]
        
        test=r.RCoT(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning,seed=2)
        pvalue=r.unlist(test)[0]
        print(pvalue)
            
    elif dtype=="mixed":
        dat_mixed=pd.concat([tmp_effect,tmp_explorer,tmp_cause,tmp_covariate],axis=1)
        dat_mixed.columns = ['col1','col2','col3','col4']
        
        mod_restriced = smf.ols('col1 ~ col3*col4 ', data = dat_mixed).fit()
        mod_unrestriced = smf.ols('col1 ~ col2*col3*col4', data = dat_mixed).fit()
        anovaResults = anova_lm(mod_restriced, mod_unrestriced)
        pvalue=anovaResults.iloc[1,5]
        print(pvalue)

    else:
        raise ValueError("dtype is not valid.")
        
    if(pvalue<=level):
        print("The Ignorability Assumption is not satisfied.")
    else:
        print("The Ignorability Assumption is satisfied.")

    return pvalue

In [ ]:
UIAT.py

In [3]:
import uiat

In [4]:
n = 1000 #changeable
c1 = 1
c2 = 1 
c3 = 1 #changeable
c4 = 1
c5 = 1
r1 = r2 = r3 = r4 = 1 #changeable
import scipy.stats as stats

#Figure 1
np.random.seed(1)

COUNT1=0 #기각 횟수
for i in range(3):

    U = np.random.normal(0,1,n) #common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+ey
    
    if i==1:
        f1_cor12=stats.pearsonr(X1, X2)[0]
        
    UIAT=uiat.UniversalIgnorabilityAssumptionTest(cause=X1,effect=Y,explorer=X2,dtype="continuous")
    pvalue = UIAT.test()
    if pvalue<=0.05:
        COUNT1 = COUNT1+1

0.27766323332439335
The Ignorability Assumption is satisfied.
0.4989722898028902
The Ignorability Assumption is satisfied.
0.2978939489606358
The Ignorability Assumption is satisfied.


### Scenario 1

In [17]:
#실제 ATE 찾아지는 상황 (but, 실제 causal effect = 0) <1종 오류>
import time

np.random.seed(1)

COUNT1 = 0
for i in range(1):
    
    n = 1000 
    k0_co = 0.8
    k1_co = 1
    k0_at = 0.8
    k1_at = 1
    k0_nt = 0.8
    k1_nt = 1

    a0 = -2 #changeable
    a1 = 0

    X = np.random.binomial(1,0.5,size=n)

    mu_Z = 1/(1+np.exp(1-2*X))
    Z = np.random.binomial(1,mu_Z,size=n)

    p_at = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_nt = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_co = 1/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))

    C=[]
    for j in range(n):
        C.append(np.random.choice(["co","at","nt"], size=1, replace=True, p=[p_at[j],p_nt[j],p_co[j]])[0])
    C=np.array(C)

    A = np.repeat(0,n)
    A[C=="at"] = 1
    A[(C=="co")*(Z==1)] = 1

    Y = np.random.normal(0,1,n)
    for k in range(n):

        if C[k]=="co":
            mu_y = k0_co+k1_co*X[k]+(a0+a1*X[k])*A[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "at":
            mu_y = k0_at+k1_at*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        else:
            mu_y = k0_nt+k1_nt*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
            
    #data = pd.DataFrame({"col1":Y,"col2":Z,"col3":A,"col4":C,"col5":X})
    #data.loc[data["col4"]=="nt","col4"] = 0 
    #data.loc[data["col4"]=="co","col4"] = 1
    #data.loc[data["col4"]=="at","col4"] = 2 
    start_time=time.time()
    pvalue = IgnorTest(cause=A, effect=Y, explorer=Z, covariate=X, level=0.05, dtype="mixed")
    print(time.time()-start_time)
    if pvalue<=0.05:
        COUNT1 = COUNT1+1   

4.262348594337282e-08
The Ignorability Assumption is not satisfied.
0.010004520416259766


In [165]:
COUNT1

999

### Scenario 2

In [174]:
# 2종 오류 (confounder인 C가 안 찾아짐.)

np.random.seed(1)

COUNT2 = 0
for i in range(1000):
    
    n = 1000 
    k0_co = 0.3
    k1_co = 1
    k0_at = 0.8
    k1_at = 1
    k0_nt = 0.3
    k1_nt = 1

    a0 = -2 #changeable
    a1 = 0

    X = np.random.binomial(1,0.5,size=n)

    mu_Z = 1/(1+np.exp(1-2*X))
    Z = np.random.binomial(1,mu_Z,size=n)

    p_at = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_nt = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_co = 1/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))

    C=[]
    for j in range(n):
        C.append(np.random.choice(["co","at","nt"], size=1, replace=True, p=[p_at[j],p_nt[j],p_co[j]])[0])
    C=np.array(C)

    A = np.repeat(0,n)
    A[C=="at"] = 1
    A[(C=="co")*(Z==1)] = 1

    Y = np.random.normal(0,1,n)
    for k in range(n):

        if C[k]=="co":
            mu_y = k0_co+k1_co*X[k]+(a0+a1*X[k])*A[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "at":
            mu_y = k0_at+k1_at*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        else:
            mu_y = k0_nt+k1_nt*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
            
    #data = pd.DataFrame({"col1":Y,"col2":Z,"col3":A,"col4":C,"col5":X})
    #data.loc[data["col4"]=="nt","col4"] = 0 
    #data.loc[data["col4"]=="co","col4"] = 1
    #data.loc[data["col4"]=="at","col4"] = 2 
    
    pvalue = IgnorTest(cause=A, effect=Y, explorer=Z, covariate=X, level=0.05, dtype="mixed")
    if pvalue<=0.01:
        COUNT2 = COUNT2+1   

8.683126104351939e-12
The Ignorability Assumption is not satisfied.
1.54037646506326e-17
The Ignorability Assumption is not satisfied.
3.188395872156527e-09
The Ignorability Assumption is not satisfied.
1.368714755154423e-12
The Ignorability Assumption is not satisfied.
1.1258324965457155e-18
The Ignorability Assumption is not satisfied.
3.2676321800335454e-07
The Ignorability Assumption is not satisfied.
3.710047742763449e-10
The Ignorability Assumption is not satisfied.
3.323845517162924e-15
The Ignorability Assumption is not satisfied.
4.44260338736169e-16
The Ignorability Assumption is not satisfied.
5.204818448208334e-12
The Ignorability Assumption is not satisfied.
6.736964755443751e-15
The Ignorability Assumption is not satisfied.
1.6155878707927935e-06
The Ignorability Assumption is not satisfied.
1.2526221519900518e-12
The Ignorability Assumption is not satisfied.
1.1094588546110155e-18
The Ignorability Assumption is not satisfied.
2.403790063679607e-15
The Ignorability Assump

In [175]:
COUNT2

1000

### Scenario 3

In [143]:
# 2종 오류 (confounder인 C가 안 찾아짐.)

np.random.seed(1)

COUNT3 = 0
for i in range(1000):
    
    n = 1000 
    k0_co = 0.3
    k1_co = 1
    k0_at = 0.8
    k1_at = 0
    k0_nt = 0.3
    k1_nt = 1

    a0 = -2 #changeable
    a1 = -1

    X = np.random.binomial(1,0.5,size=n)

    mu_Z = 1/(1+np.exp(1-2*X))
    Z = np.random.binomial(1,mu_Z,size=n)

    p_at = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_nt = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_co = 1/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))

    C=[]
    for j in range(n):
        C.append(np.random.choice(["co","at","nt"], size=1, replace=True, p=[p_at[j],p_nt[j],p_co[j]])[0])
    C=np.array(C)

    A = np.repeat(0,n)
    A[C=="at"] = 1
    A[(C=="co")*(Z==1)] = 1

    Y = np.random.normal(0,1,n)
    for k in range(n):

        if C[k]=="co":
            mu_y = k0_co+k1_co*X[k]+(a0+a1*X[k])*A[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "at":
            mu_y = k0_at+k1_at*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        else:
            mu_y = k0_nt+k1_nt*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
            
    #data = pd.DataFrame({"col1":Y,"col2":Z,"col3":A,"col4":C,"col5":X})
    #data.loc[data["col4"]=="nt","col4"] = 0 
    #data.loc[data["col4"]=="co","col4"] = 1
    #data.loc[data["col4"]=="at","col4"] = 2 
    
    pvalue = IgnorTest(cause=A, effect=Y, explorer=Z, covariate=X, level=0.05, dtype="mixed")
    if pvalue<=0.05:
        COUNT3 = COUNT3+1   

8.683126104351939e-12
The Ignorability Assumption is not satisfied.
1.54037646506326e-17
The Ignorability Assumption is not satisfied.
3.1883958721572183e-09
The Ignorability Assumption is not satisfied.
1.3687147551545835e-12
The Ignorability Assumption is not satisfied.
1.1258324965457795e-18
The Ignorability Assumption is not satisfied.
3.2676321800335454e-07
The Ignorability Assumption is not satisfied.
3.710047742763449e-10
The Ignorability Assumption is not satisfied.
3.323845517162924e-15
The Ignorability Assumption is not satisfied.
4.442603387360144e-16
The Ignorability Assumption is not satisfied.
5.204818448207724e-12
The Ignorability Assumption is not satisfied.
6.736964755443751e-15
The Ignorability Assumption is not satisfied.
1.6155878707929714e-06
The Ignorability Assumption is not satisfied.
1.2526221519901941e-12
The Ignorability Assumption is not satisfied.
1.1094588546110785e-18
The Ignorability Assumption is not satisfied.
2.403790063679607e-15
The Ignorability Ass

In [144]:
COUNT3

1000

### Scenario 4

In [154]:
# 2종 오류 (confounder인 C가 안 찾아짐.)

np.random.seed(1)

COUNT4 = 0
for i in range(1000):
    
    n = 1000 
    k0_co = 0.3
    k1_co = 1
    k0_at = 1.5
    k1_at = 1
    k0_nt = -1
    k1_nt = 2

    a0 = -2 #changeable
    a1 = -1

    X = np.random.binomial(1,0.5,size=n)

    mu_Z = 1/(1+np.exp(1-2*X))
    Z = np.random.binomial(1,mu_Z,size=n)

    p_at = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_nt = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_co = 1/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))

    C=[]
    for j in range(n):
        C.append(np.random.choice(["co","at","nt"], size=1, replace=True, p=[p_at[j],p_nt[j],p_co[j]])[0])
    C=np.array(C)

    A = np.repeat(0,n)
    A[C=="at"] = 1
    A[(C=="co")*(Z==1)] = 1

    Y = np.random.normal(0,1,n)
    for k in range(n):

        if C[k]=="co":
            mu_y = k0_co+k1_co*X[k]+(a0+a1*X[k])*A[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "at":
            mu_y = k0_at+k1_at*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        else:
            mu_y = k0_nt+k1_nt*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
            
    #data = pd.DataFrame({"col1":Y,"col2":Z,"col3":A,"col4":C,"col5":X})
    #data.loc[data["col4"]=="nt","col4"] = 0 
    #data.loc[data["col4"]=="co","col4"] = 1
    #data.loc[data["col4"]=="at","col4"] = 2 
    
    pvalue = IgnorTest(cause=A, effect=Y, explorer=Z, covariate=X, level=0.05, dtype="mixed")
    if pvalue<=0.01:
        COUNT4 = COUNT4+1   

1.3081458486699698e-21
The Ignorability Assumption is not satisfied.
8.029375791169955e-26
The Ignorability Assumption is not satisfied.
4.2544548440813966e-17
The Ignorability Assumption is not satisfied.
1.2902411302522566e-19
The Ignorability Assumption is not satisfied.
4.7247695517117026e-29
The Ignorability Assumption is not satisfied.
1.308726267365342e-12
The Ignorability Assumption is not satisfied.
2.9151474115202787e-19
The Ignorability Assumption is not satisfied.
7.971545142575393e-24
The Ignorability Assumption is not satisfied.
2.8748649129491176e-25
The Ignorability Assumption is not satisfied.
6.945270726561645e-18
The Ignorability Assumption is not satisfied.
3.358352145587968e-23
The Ignorability Assumption is not satisfied.
1.065730421570571e-11
The Ignorability Assumption is not satisfied.
8.506965386616636e-19
The Ignorability Assumption is not satisfied.
4.223158546021412e-26
The Ignorability Assumption is not satisfied.
5.440485105258697e-21
The Ignorability Ass

In [155]:
COUNT4

1000

### Scenario 5

In [12]:
#실제 ATE 찾아지는 상황 (but, 실제 causal effect = 0) <1종 오류>

np.random.seed(1)

COUNT5 = 0
for i in range(1000):
    
    n = 1000 
    k0_co = 0.8
    k1_co = 1
    k0_at = 0.8
    k1_at = 1
    k0_nt = 0.8
    k1_nt = 1
    k0_de = 0.8
    k1_de = 1

    a0 = 0 #changeable
    a1 = 0

    X = np.random.binomial(1,0.5,size=n)

    mu_Z = 1/(1+np.exp(1-2*X))
    Z = np.random.binomial(1,mu_Z,size=n)

    p_at = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_nt = np.exp(-2.5+3.5*X)/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_co = 0.75/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    p_de = 0.25/(1+np.exp(-2.5+3.5*X)+np.exp(-2.5+3.5*X))
    

    C=[]
    for j in range(n):
        C.append(np.random.choice(["co","at","nt","de"], size=1, replace=True, p=[p_at[j],p_nt[j],p_co[j],p_de[j]])[0])
    C=np.array(C)

    A = np.repeat(0,n)
    A[C=="at"] = 1
    A[(C=="co")*(Z==1)] = 1
    A[(C=="de")*(Z==0)] = 1

    Y = np.random.normal(0,1,n)
    for k in range(n):

        if C[k]=="co":
            mu_y = k0_co+k1_co*X[k]+(a0+a1*X[k])*A[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "at":
            mu_y = k0_at+k1_at*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        elif C[k] == "nt":
            mu_y = k0_nt+k1_nt*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
        else:
            mu_y = k0_de+k1_de*X[k]
            Y[k] = np.random.normal(mu_y,1,1)[0]
            
    #data = pd.DataFrame({"col1":Y,"col2":Z,"col3":A,"col4":C,"col5":X})
    #data.loc[data["col4"]=="nt","col4"] = 0 
    #data.loc[data["col4"]=="co","col4"] = 1
    #data.loc[data["col4"]=="at","col4"] = 2 
    
    pvalue = IgnorTest(cause=A, effect=Y, explorer=Z, covariate=X, level=0.05, dtype="mixed")
    if pvalue<=0.05:
        COUNT5 = COUNT5+1   

0.15045047478034115
The Ignorability Assumption is satisfied.
0.12230630995226696
The Ignorability Assumption is satisfied.
0.6842562217271986
The Ignorability Assumption is satisfied.
0.3849341996125172
The Ignorability Assumption is satisfied.
0.7864708419144251
The Ignorability Assumption is satisfied.
0.19984731731957814
The Ignorability Assumption is satisfied.
0.07640023285783452
The Ignorability Assumption is satisfied.
0.3674828757835217
The Ignorability Assumption is satisfied.
0.4212027919809933
The Ignorability Assumption is satisfied.
0.617749348943258
The Ignorability Assumption is satisfied.
0.8165009251419331
The Ignorability Assumption is satisfied.
0.23396256733228243
The Ignorability Assumption is satisfied.
0.3320562732491292
The Ignorability Assumption is satisfied.
0.09430945095192358
The Ignorability Assumption is satisfied.
0.3319392810555479
The Ignorability Assumption is satisfied.
0.31477865589164505
The Ignorability Assumption is satisfied.
0.4803066809038967

In [13]:
COUNT5

43

In [15]:
import time
time.time()

1633413010.7920482

In [16]:
time.time()

1633413017.3669047